In [ ]:
from azureml.core import Workspace, Experiment

# Configure workspace and experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="titanic-optimization")

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# cluster configuration
cluster_name = "mldemocompute"
min_nodes = 0
max_nodes = 4
vm_size = "STANDARD_D2_V2"

# verify that the cluster exists already
try:
    aml_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    print('Cluster not `%s` not found, creating one now.' % cluster_name)
    config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes=min_nodes, max_nodes=max_nodes)
    aml_cluster = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=config)

# wait until the cluster is ready
aml_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
 
def get_env(packages=None):
    packages = packages or []
    packages += ['azureml-defaults']
    env = Environment(name="tf")
    env.python.conda_dependencies = CondaDependencies.create(pip_packages=packages)
    return env

In [ ]:
tf_env = get_env(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [ ]:
from azureml.train.hyperdrive import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import *

In [ ]:
from azureml.train.hyperdrive import GridParameterSampling

grid_sampling = GridParameterSampling({
    "--first-layer-neurons": choice(16, 32, 64, 128),
    "--second-layer-neurons": choice(16, 32, 64, 128),
    "--batch-size": choice(16, 32)
})

In [ ]:
primary_metric_name = "accuracy"
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

In [ ]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory="train", 
                      script="train.py", 
                      environment = tf_env,
                      compute_target = aml_cluster)

In [ ]:
max_total_runs = 10
max_concurrent_runs = 4

In [ ]:
from azureml.train.hyperdrive import HyperDriveConfig

hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=grid_sampling, 
    primary_metric_name=primary_metric_name, 
    primary_metric_goal=primary_metric_goal,
    max_total_runs=max_total_runs,
    max_concurrent_runs=max_concurrent_runs)

In [ ]:
hyperdrive_run = exp.submit(hyperdrive_run_config)
print(hyperdrive_run.get_portal_url())

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=False)

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import *

random_sampling = RandomParameterSampling({
    "--learning-rate": uniform(0.01, 0.1),
    "--momentum": normal(0.75, 0.25),
})

In [ ]:
truncation_percentage = 10
evaluation_interval = 5
delay_evaluation = 10

In [ ]:
from azureml.train.hyperdrive import TruncationSelectionPolicy

early_termination_policy = TruncationSelectionPolicy(
   truncation_percentage=truncation_percentage,
   evaluation_interval=evaluation_interval,
   delay_evaluation=delay_evaluation)

In [ ]:
max_total_runs = 10
max_duration_minutes = 60

In [ ]:
from azureml.train.hyperdrive import HyperDriveConfig

hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=random_sampling, 
    primary_metric_name=primary_metric_name, 
    primary_metric_goal=primary_metric_goal,
    policy=early_termination_policy,
    max_total_runs=max_total_runs,
    max_duration_minutes=max_duration_minutes)

In [ ]:
hyperdrive_run = exp.submit(hyperdrive_run_config)
print(hyperdrive_run.get_portal_url())

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=False)